# TCC Residência Ciência de Dados - NLP (Similaridade de Documentos)

- Lucas Natan Correia Couri - lncc2@cin.ufpe.br

# Importando bibliotecas

In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

# Carregando conjunto de dados

In [2]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\lucas\AppData\Local\Temp/ipykernel_12256/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [3]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [4]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
#df_text.head(1)

In [5]:
df_text.shape

(1357, 4)

# Pré-processamento dos dados

In [6]:
pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True},
    {"basic_processing": True, "stemming": True},
    {"basic_processing": True, "no_stopwords": True, "stemming": True},
    {"basic_processing": True, "lema":  True},
    {"basic_processing": True, "no_stopwords": True, "lema": True}]

df_pp = pp.pre_process_all(df_text, pre_processing_list)
#df_pp = pp.pre_process_all(df_text.iloc[:3], pre_processing_list)

In [7]:
#df_pp.head(1)

# Algoritmos

In [7]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## Bag of Words (CountVectorizer)

In [8]:
(df_bow, time_bow) = nlp.apply_bow(df_pp, len(pre_processing_list))

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [10]:
df_bow.to_csv('./resultados/bow.csv', index = False)
time_bow.to_csv('./resultados/bow_time.csv', index = False)

In [11]:
df_results = df_results.join(df_bow)

In [12]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636


## TF-IDF

In [13]:
(df_tf_idf, time_tf_idf) = nlp.apply_tf_idf(df_pp, len(pre_processing_list))

In [14]:
df_tf_idf.to_csv('./resultados/tf_idf.csv', index = False)
time_tf_idf.to_csv('./resultados/tf_idf_time.csv', index = False)

In [15]:
df_results = df_results.join(df_tf_idf)

In [16]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,tf_idf1,tf_idf2,tf_idf3,tf_idf4,tf_idf5,tf_idf6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,0.283060,0.070444,0.288851,0.069445,0.281601,0.067110
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,0.357191,0.045734,0.347028,0.045691,0.352090,0.044953
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,0.405624,0.239488,0.421826,0.263690,0.463674,0.305044


## BERT

In [12]:
pre_processing_list

[{},
 {'basic_processing': True},
 {'basic_processing': True, 'no_stopwords': True},
 {'basic_processing': True, 'stemming': True},
 {'basic_processing': True, 'no_stopwords': True, 'stemming': True},
 {'basic_processing': True, 'lema': True},
 {'basic_processing': True, 'no_stopwords': True, 'lema': True}]

In [15]:
df_pp.columns

df_pp[['doc2_pipeline0', 'Overall']]

,doc2_pipeline0,Overall
0,"[PORT-AU-PRINCE, ,, Haiti, —, Haitian, President, Jovenel, Moïse, broke, with, tradition, on, Wednesday, and, celebrated, the, country, ’, s, independence, day, in, the, capital, for, security, reasons, following, months, of, political, turmoil, ., Moïse, ,, whose, government, has, been, accused, of, corruption, ,, denounced, graft, during, his, speech, at, the, National, Palace, in, Port-au-Prince, and, urged, Haiti, ’, s, elite, to, work, with, the, government, and, help, create, employment, ., “, We, ’, re, still, extremely, poor, ,, ”, he, said, ., “, Those, who, continue, to, get, rich, find, it, normal, that, they, do, not, pay, taxes, ,, find, ...]",4.000000
1,"[BERLIN, -, A, fire, at, a, zoo, in, western, Germany, in, the, first, minutes, of, 2020, killed, more, than, 30, animals, ,, including, apes, ,, monkeys, ,, bats, and, birds, ,, authorities, said, ., Police, said, the, fire, may, have, been, caused, by, sky, lanterns, launched, to, celebrate, the, new, year, ., Several, witnesses, reported, that, they, had, seen, the, cylindrical, paper, lanterns, with, little, fires, inside, flying, in, the, night, sky, shortly, after, midnight, Wednesday, near, the, Krefeld, zoo, ,, Gerd, Hoppmann, ,, the, city, ’, s, head, of, criminal, police, told, reporters, ., “, People, reported, seeing, those, ...]",3.666667
2,"[PALM, BEACH, ,, United, States, —, US, President, Donald, Trump, on, Tuesday, said, he, does, not, foresee, war, with, Tehran, ,, after, pro-Iranian, demonstrators, stormed, the, US, embassy, in, Iraq, ., “, I, don, ’, t, see, that, happening, ,, ”, Trump, said, at, his, holiday, retreat, in, Florida, when, a, reporter, asked, about, the, possibility, of, war, with, the, Islamic, republic, ., “, I, like, peace, ,, ”, the, president, said, before, heading, in, to, New, Year, ’, s, celebrations, ., Get, The, Times, of, Israel, 's, Daily, Edition, by, email, and, never, miss, our, top, stories, Free, Sign, ...]",2.333333
3,"[Rapid, digitisation, and, growth, in, both, online, buyer, base, and, spending, will, help, India, 's, online, food, industry, to, become, an, $, 8, billion, (, roughly, Rs, ., 56,000, ), market, by, 2022, --, growing, at, a, CAGR, of, 25-30, percent, ,, a, new, report, said, on, Tuesday, ., The, report, by, Google, and, Boston, Consulting, Group, (, BCG, ), revealed, that, variety, in, cuisines, (, 35, percent, ), was, one, of, the, top, reasons, for, recurrent, use, of, online, food, ordering, apps, ,, followed, by, good, discounts, and, convenience, ., ``, Food, tech, has, now, made, its, presence, in, ...]",2.000000
4,"[BANGALORE, :, India, plans, to, make, a, fresh, attempt, to, land, an, unmanned, mission, on, the, moon, in, 2020, after, a, failed, bid, last, year, ,, the, head, of, the, country, 's, space, programme, said, on, Wednesday, (, Jan, 1, ), ., Work, is, going, ``, smoothly, '', on, the, Chandrayaan-3, mission, to, put, a, rover, probe, on, the, moon, 's, surface, ,, Indian, Space, Research, Organisation, chairman, K., Sivan, told, a, press, conference, ., Advertisement, Advertisement, '', We, are, targeting, the, launch, for, this, year, but, it, may, spillover, to, next, year, ,, '', Sivan, said, ., Indian, sources, ...]",1.250000
...,...,...
1352,"[JOHANNESBURG, -, A, manhunt, is, underway, after, two, women, were, killed, and, six, others, wounded, in, a, drive-by, shooting, in, Melville, ,, Johannesburg, ., The, shooting, took, place, just, after, midnight, at, a, restaurant, ., Police, said, in, a, statement, that, a, vehicle, approached, the, restaurant, and, fired, multiple, shots, at, patrons, sitting, outside, ., The, two, victims, are, aged, between, 30, and, 40, ., The, wounded, were, rushed, to, nearby, hospitals, ., Police, said, that, two, of, the, victims, are, in, critical, condition, ., Gauteng, police, commissioner, Lieutenant-General, Elias, Mawela, has, condemned, the, incident, ., Mawela, urged, the, investigators, ...]",1.000000

In [17]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
model_name = 'base_mean'

(df_bert, time_bert) = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

In [18]:
df_bert.to_csv('./resultados/bert_mean.csv', index = False)
time_bert.to_csv('./resultados/bert_mean_time.csv', index = False)

In [19]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-max-tokens')
model_name = 'base_max'

(df_bert, time_bert) = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

In [20]:
df_bert.to_csv('./resultados/bert_max.csv', index = False)
time_bert.to_csv('./resultados/bert_max_time.csv', index = False)

In [21]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_name = 'miniLM'

(df_bert, time_bert) = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

In [22]:
df_bert.to_csv('./resultados/bert_MiniLM.csv', index = False)
time_bert.to_csv('./resultados/bert_MiniLM_time.csv', index = False)

In [23]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,bert_base_max4,bert_base_max5,bert_base_max6,bert_miniLM0,bert_miniLM1,bert_miniLM2,bert_miniLM3,bert_miniLM4,bert_miniLM5,bert_miniLM6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,...,0.811856,0.855984,0.875019,0.120794,0.094716,0.135753,0.178856,0.283669,0.098491,0.128578
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,...,0.850374,0.887332,0.887321,0.141600,0.197086,0.170077,0.292377,0.312857,0.222857,0.178241
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,...,0.940309,0.933166,0.953908,0.660645,0.575526,0.475314,0.535299,0.564201,0.595904,0.468557


## Doc2vec

In [24]:
nlp.train_doc2vec(df_pp, len(pre_processing_list))

In [25]:
(df_doc2vec, time_doc2vec) = nlp.apply_doc2vec(df_pp, len(pre_processing_list))

In [26]:
df_doc2vec.to_csv('./resultados/doc2vec.csv', index = False)
time_doc2vec.to_csv('./resultados/doc2vec_time.csv', index = False)

In [27]:
df_results = df_results.join(df_doc2vec)

In [28]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,bert_miniLM4,bert_miniLM5,bert_miniLM6,doc2vec0,doc2vec1,doc2vec2,doc2vec3,doc2vec4,doc2vec5,doc2vec6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,...,0.283669,0.098491,0.128578,0.806179,0.830591,0.955678,0.807286,0.923678,0.800442,0.962045
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,...,0.312857,0.222857,0.178241,0.845838,0.849177,0.927025,0.831141,0.916924,0.830910,0.876261
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,...,0.564201,0.595904,0.468557,0.909131,0.886117,0.918790,0.890987,0.911085,0.884283,0.921056


## Word2vec

In [29]:
import gensim.downloader as api

In [30]:
model = api.load('word2vec-google-news-300')

In [31]:
(df_word2vec, time_word2vec) = nlp.apply_word2vec(df_pp, len(pre_processing_list), model)

In [32]:
df_word2vec.to_csv('./resultados/word2vec.csv', index = False)
time_word2vec.to_csv('./resultados/word2vec_time.csv', index = False)

In [33]:
df_results = df_results.join(df_word2vec)

In [34]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,doc2vec4,doc2vec5,doc2vec6,word2vec0,word2vec1,word2vec2,word2vec3,word2vec4,word2vec5,word2vec6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,...,0.923678,0.800442,0.962045,0.998293,0.998228,0.997522,0.997583,0.996466,0.998236,0.997508
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,...,0.916924,0.830910,0.876261,0.998788,0.999105,0.998646,0.998335,0.997295,0.999010,0.998614
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,...,0.911085,0.884283,0.921056,0.999040,0.999128,0.998001,0.999196,0.998049,0.999311,0.998574


In [35]:
time_word2vec.head()

,0,1
0,word2vec0,112.764316
1,word2vec1,92.309248
2,word2vec2,54.954898
3,word2vec3,92.435505
4,word2vec4,53.350397


## Resultados

In [42]:
df_results = df_pp[["pair_id", "Overall"]]

In [47]:
df_list = []
paths = ["./resultados/bow.csv", "./resultados/tf_idf.csv", "./resultados/bert_mean.csv",
         "./resultados/bert_max.csv", "./resultados/bert_MiniLM.csv", "./resultados/word2vec.csv", 
         "./resultados/doc2vec.csv"]
for path in paths:
    df_list.append(pd.read_csv(path))


df_results = df_results.join(df_list)
#df_results = pd.concat(df_list)
df_results.head()

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,word2vec4,word2vec5,word2vec6,doc2vec0,doc2vec1,doc2vec2,doc2vec3,doc2vec4,doc2vec5,doc2vec6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,...,0.996466,0.998236,0.997508,0.806179,0.830591,0.955678,0.807286,0.923678,0.800442,0.962045
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,...,0.997295,0.999010,0.998614,0.845838,0.849177,0.927025,0.831141,0.916924,0.830910,0.876261
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,...,0.998049,0.999311,0.998574,0.909131,0.886117,0.918790,0.890987,0.911085,0.884283,0.921056


In [46]:
df_time_list = []
paths = ["./resultados/bow_time.csv", "./resultados/tf_idf_time.csv", "./resultados/bert_mean_time.csv",
         "./resultados/bert_max_time.csv", "./resultados/bert_MiniLM_time.csv", "./resultados/word2vec_time.csv", 
         "./resultados/doc2vec_time.csv"]
for path in paths:
    df_time_list.append(pd.read_csv(path))


#df_time_results = df_results.join(df_time_list)
df_time_results = pd.concat(df_time_list)

,0,1
0,bow0,0.008486
1,bow1,0.005985
2,bow2,0.005984
3,bow3,0.006012
4,bow4,0.005956
5,bow5,0.006980
6,bow6,0.005984
0,tf_idf0,0.006952
1,tf_idf1,0.006981
2,tf_idf2,0.004018


In [50]:
df_time_results.head(20)

,0,1
0,bow0,0.008486
1,bow1,0.005985
2,bow2,0.005984
3,bow3,0.006012
4,bow4,0.005956
5,bow5,0.006980
6,bow6,0.005984
0,tf_idf0,0.006952
1,tf_idf1,0.006981
2,tf_idf2,0.004018


In [48]:
df_results.head()

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,word2vec4,word2vec5,word2vec6,doc2vec0,doc2vec1,doc2vec2,doc2vec3,doc2vec4,doc2vec5,doc2vec6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.269091,...,0.996466,0.998236,0.997508,0.806179,0.830591,0.955678,0.807286,0.923678,0.800442,0.962045
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.358770,...,0.997295,0.999010,0.998614,0.845838,0.849177,0.927025,0.831141,0.916924,0.830910,0.876261
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.395518,...,0.998049,0.999311,0.998574,0.909131,0.886117,0.918790,0.890987,0.911085,0.884283,0.921056


In [ ]:
#df_results.corr()["Overall"]
df_results.corr()